In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_train = pd.read_csv('train_5CLrC8b/train.csv')
df_test = pd.read_csv('test.csv')

In [ ]:
df_train['source_'] = 1
df_test['source_'] =0
df = pd.concat([df_train,df_test])
df.reset_index(inplace=True)
df.drop(['index'],inplace=True,axis=1)
df.head()

In [ ]:
a = list(df['season_holidayed_code'].isnull())
extra_shc = []
for i in a:
    if i == False:
        extra_shc.append(1)
    else:
        extra_shc.append(0)

In [ ]:
b = list(df['state_code_residence'].isnull())
extra_scr = []
for i in b:
    if i == False:
        extra_scr.append(1)
    else:
        extra_scr.append(0)

In [ ]:
df['scr'] = b
df['shc'] = a

In [ ]:
df.fillna(0,inplace=True,axis=0)

In [ ]:
df.head()

In [ ]:
dte = pd.to_datetime(df['checkin_date'],dayfirst=True)
dte1 = pd.to_datetime(df['checkout_date'],dayfirst=True)
dte2 = pd.to_datetime(df['booking_date'],dayfirst=True)

In [ ]:
year = dte.apply(lambda x : x.year)
month = dte.apply(lambda x : x.month)
day = dte.apply(lambda x : x.day)

year_1 = dte1.apply(lambda x : x.year)
month_1 = dte1.apply(lambda x : x.month)
day_1 = dte1.apply(lambda x : x.day)

year_2 = dte2.apply(lambda x : x.year)
month_2 = dte2.apply(lambda x : x.month)
day_2 = dte2.apply(lambda x : x.day)


In [ ]:
df['month']=month
df['year'] = year
df['day'] = day

df['month_1']=month_1
df['year_1'] = year_1
df['day_1'] = day_1


df['month_2']=month_2
df['year_2'] = year_2
df['day_2'] = day_2



In [ ]:
seasons = []
for i in list(df['month']):
    if i >=1 and i<4:
        seasons.append('w')
    if i>=4 and i<8:
        seasons.append('q')
    if i>=8 and i<11:
        seasons.append('e')
    if i>=11 and i<=12:
        seasons.append('t')
df['ses']= seasons

In [ ]:
df.drop(['checkout_date','checkin_date','booking_date','memberid','reservation_id'],inplace=True,axis=1)

In [ ]:
# import category_encoders as ce
# l = ['booking_type_code',
#        'channel_code', 'cluster_code', 'main_product_code',
#        'member_age_buckets','resort_id','state_code_residence',
#        'resort_region_code', 'resort_type_code', 'room_type_booked_code',
#         'persontravellingid','reservationstatusid_code',
#         'state_code_resort','season_holidayed_code','scr','shc','ses']
# encoder = ce.()
# df[l] = encoder.fit_transform(df[l])
# #df_ = pd.get_dummies(df,columns=l)

In [ ]:
from sklearn.preprocessing import LabelBinarizer
b_data = []
l = ['booking_type_code',
       'channel_code', 'cluster_code', 'main_product_code',
       'member_age_buckets','resort_id','state_code_residence',
       'resort_region_code', 'resort_type_code', 'room_type_booked_code',
        'persontravellingid','reservationstatusid_code',
        'state_code_resort','season_holidayed_code','scr','shc']

import category_encoders as ce


# Specify the columns to encode then fit and transform
encoder = ce.ordinal.OrdinalEncoder(cols=l)
df= encoder.fit_transform(df, verbose=1)


In [ ]:
df.drop(['ses'],inplace=True,axis=1)

In [ ]:
preprocessed_main_test = df.loc[df['source_'] == 0]
preprocessed_main_train = df.loc[df['source_'] == 1]

In [ ]:
preprocessed_main_test.drop(['source_','amount_spent_per_room_night_scaled'],axis =1,inplace=True)

In [ ]:
y_train = preprocessed_main_train['amount_spent_per_room_night_scaled']
x_train = preprocessed_main_train.drop(['amount_spent_per_room_night_scaled','source_'],axis =1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.linear_model import LogisticRegression

In [ ]:
import lightgbm as lgb
all_pred = []
# params = {
# 'num_leaves': 30,
# 'max_bin': 50,
# 'min_data_in_leaf': 50,
# 'learning_rate': 0.01,
# 'min_sum_hessian_in_leaf': 0.000446,
# 'bagging_fraction': 0.60,
# 'bagging_freq': 15,
# 'max_depth': 30,
# 'save_binary': True,
# 'seed': 31452,
# 'feature_fraction_seed': 31415,
# 'feature_fraction': 0.51,
# 'bagging_seed': 31415,
# 'drop_seed': 31415,
# 'data_random_seed': 31415,
# 'objective': 'regression',
# 'boosting_type': 'gbdt',
# 'verbose': 1,
# 'metric': 'rmse',
# 'is_unbalance': False,
# #     'categorical_feature': "name:branch_id"
# # 'scale_pos_weight': len(train_df["target"] == 0) / sum(train_df["target"])
# }
 
# X_train, X_test, y_train_internal, y_test_internal = train_test_split(x_train,y_train ,test_size =.15)

# d_train = lgb.Dataset(X_train, label=y_train_internal)
# clf = lgb.train(params, d_train,30000)
#for i in range(1,2):
from sklearn.feature_selection import RFE
lg =  lgb.LGBMRegressor(n_estimators=2000)
X_train, X_test, y_train_internal, y_test_internal = train_test_split(x_train,y_train ,test_size =.15)

#rfe = RFE(lg, 30)
#fit = rfe.fit(X_train, y_train_internal)


lg.fit(X_train, y_train_internal,eval_set=(X_test,y_test_internal),eval_metric='rmse',early_stopping_rounds=100,verbose=0)
pred = lg.predict(preprocessed_main_test)
# all_pred.append(pred)

In [ ]:
fit.predict()

In [ ]:
pred = lg.predict(preprocessed_main_test)
pred

In [ ]:
# import numpy as np
# predd = np.array(all_model_pred).mean(axis =0)
# len(predd)

In [ ]:
p = (pred)

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
pred_orig_test = lg.predict(X_test)
rms = sqrt(mean_squared_error(y_test_internal, pred_orig_test))
rms

In [ ]:
#pred = regr.predict(preprocessed_main_test)
final_res = list(zip(df_test['reservation_id'],pred))

In [ ]:
res_ = pd.DataFrame(final_res,columns=['reservation_id','amount_spent_per_room_night_scaled'])

In [ ]:
res_.to_csv('sub_13_poly_enc.csv',index=False)

In [ ]:
lg.feature_importances_

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline
# sorted(zip(clf.feature_importances_, X.columns), reverse=True)
feature_imp = pd.DataFrame(sorted(zip(lg.feature_importances_,x_train.columns)), columns=['Value','Feature'])

plt.figure(figsize=(20, 30))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()
plt.savefig('lgbm_importances-01.png')


In [ ]:
set(df_train['memberid'].unique()).intersection(set(df_test['memberid'].unique()))